# Démo : Utiliser l'API Google Sheets

Ce notebook présente les principales fonctions et méthodes de l'API Google Sheets, qui sera utilisée dans le use case APIs.

Pour pouvoir exécuter les cellules, il est nécessaire d'avoir configuré un compte de service Google Cloud - n'hésitez pas à télécharger le PDF d'instructions.

Nous utiliserons l'API wrapper `gspread` (Google Spreadsheets), dont la documentation est disponible [ici](https://gspread.readthedocs.io/en/latest/user-guide.html).

### Installation

In [ ]:
!pip install gspread 
!pip install oauth2client

In [ ]:
import gspread

### Connexion à l'API Google Sheets

In [ ]:
# Copier-coller le chemin de la clé téléchargée lors de la création du compte
# Cela devrait ressembler à : 
path = "/Users/Thomas/Downloads/databird-309423-c4ff87ee5aec.json"

In [ ]:
# Authentification / Création du client
gc = gspread.service_account(filename=path)

## Manipuler les fichiers Google Sheets

### Ouvrir un fichier Google Sheets

Il est possible d'ouvrir une spreadsheet par son nom, sa clé (visible dans l'URL du spreadsheet) ou bien son url complète. 

Dans cet exemple, j'utilise un fichier vide nomme "Twitter Analysis" ([lien](https://docs.google.com/spreadsheets/d/1c_Eij3Y-fAX7qaOQKSFOBivtV5Zb3oNprY3qE_PiNZU/edit#gid=0)), que j'ai auparavant créé et que j'utiliserai dans le use case APIs. Vous pouvez le copier afin d'avoir un fichier similaire.

In [ ]:
file = gc.open_by_url('https://docs.google.com/spreadsheets/d/1c_Eij3Y-fAX7qaOQKSFOBivtV5Zb3oNprY3qE_PiNZU/edit#gid=0')
file = gc.open_by_key('1c_Eij3Y-fAX7qaOQKSFOBivtV5Zb3oNprY3qE_PiNZU')
#file = gc.open('twitter-analysis')
file

### Sélectionner une feuille de calcul

Une fois le fichier ouvert, nous avons besoin de sélectionner l'une des feuilles de calcul, soit par son titre, soit par son numéro d'index, soit avec l'attribut `.sheet1` (nom par défaut de la première feuille de calcul).

In [ ]:
worksheet = file.sheet1
worksheet = file.get_worksheet(0)
#worksheet = file.worksheet("Statistics")
worksheet

In [ ]:
# Tout d'un seul coup
worksheet = gc.open_by_key('1c_Eij3Y-fAX7qaOQKSFOBivtV5Zb3oNprY3qE_PiNZU').sheet1

### BONUS : Créer / partager / supprimer  un fichier et une feuille de calcul

Le package nous permet également de créer et de partager un nouveau Google Sheets programmatiquement. Dans ce fichier, nous pouvons ajouter ou supprimer des feuilles de calcul. Vous pouvez vérifier que tout fonctionne au fur et à mesure dans votre compte Google !

In [ ]:
# Créer une spreadsheet vide
new_file = gc.create('A new spreadsheet')

In [ ]:
# Partager une spreadsheet
new_file.share('contact@data-bird.co', perm_type='user', role='writer')

In [ ]:
# Ajouter une feuille de calcul
new_worksheet = file.add_worksheet(title="A new worksheet", rows="100", cols="20")

file.worksheets()

In [ ]:
# Supprimer une feuille de calcul
file.del_worksheet(new_worksheet)

file.worksheets()

## Manipuler les cellules d'un fichier Google Sheets

### Modifier les cellules et les lignes

Pour modifier une cellule ou plage de cellule, nous pouvons utiliser la méthode  `.update()`, en précisant les coordonnées des cellules à modifier.

In [ ]:
# Modifier cellule individuelle
worksheet.update('D2', 'Bingo!')

In [ ]:
# Modifier une plage de cellules
worksheet.update('A2:B3', [[1, 2], [3, 4]])

La **méthode que nous utiliserons le plus** dans le cas pratique de ce module est d'**ajouter une ligne à la fin d'une feuille** : `.append_row()`, qui prend en argument les données à ajouter sous forme de **liste**.

In [ ]:
test = list(range(13))

In [ ]:
worksheet.append_row(test)

### BONUS : Extraire les valeurs des cellules

On extrait les données d'une cellule avec `acell()` ou `cell()` et la méthode `.value`. Pour récupérer toute une ligne ou colonne, la méthode sera :  `.row_values()` ou `col_values()`.

Remarque : on peut ajouter le paramètre  `value_render_option='FORMULA'` dans  `acell()` ou `cell()` pour récupérer la formule de la cellule plutôt que sa valeur.

In [ ]:
b1 = worksheet.acell('B1').value
b1 = worksheet.cell(1, 2).value
b1

In [ ]:
first_row = worksheet.row_values(1)
first_col = worksheet.col_values(1)
first_row

Enfin, il existe une méthode pour recréer un dataframe à partir d'une feuille Google Sheets :

In [ ]:
import pandas as pd

df = pd.DataFrame(worksheet.get_all_records())
df

Beaucoup d'autres commandes existent. En particulier, on peut rechercher les cellules contenant un texte spécifique avec `.find()` et `.findall()`, et on peut formatter les cellules avec `.format()`.

In [ ]:
# Formatage de l'en-tête 
worksheet.format('A1:M1', {'textFormat': {'italic': True}})

N'hésitez pas à lire la [documentation](https://gspread.readthedocs.io/en/latest/user-guide.html) pour en apprendre plus ! 